<p style='
  color: #3b4045; 
  text-align: center;
  font-weight: bold;
  font-family: -apple-system,BlinkMacSystemFont, "Segoe UI Adjusted","Segoe UI","Liberation Sans",sans-serif;     font-size: 2.07692308rem; '> 
    Pretrained Segmentation Model
</p>


In [4]:
import cv2
from ipywidgets import GridspecLayout
from jppype import imshow, sync_views

from fundus_vessels_toolkit.segment import segment_vessels
from fundus_vessels_toolkit.utils.image import load_img

In [5]:
# Path to the raw fundus image
RAW_PATH = "PATH/TO/FUNDUS/IMAGE"

# Path to a vessel ground truth (for comparison purposes)
VESSELS_PATH = "PATH/TO/VESSEL/GROUND/TRUTH"

### Load Image


In [6]:
raw = load_img(RAW_PATH)
vessels_true = load_img(VESSELS_PATH, binarize=True)

### Segment vessels


When called, `fundus_vessels_toolkit.models.segment()` will download the weights of a pretrained model, instanciate the model, convert the provided image to the appropriate tensor format, pass it to the model and return the prediction converted back to numpy format.


In [7]:
vessels = segment_vessels(raw)

In [9]:
from fundus_vessels_toolkit.seg_to_graph.graph_parsing import fix_skeleton
from skimage.morphology import skeletonize as skimage_skeletonize
from fundus_vessels_toolkit.seg_to_graph.skeletonization import skeletonize


def fast_skeletonize(vessels):
    skel = skimage_skeletonize(vessels, method="lee")
    return fix_skeleton(skel.astype(bool))


%time skeleton = skeletonize(vessels)
%time fast_skeleton = fast_skeletonize(vessels)

CPU times: user 115 ms, sys: 3.05 ms, total: 118 ms
Wall time: 117 ms
CPU times: user 263 ms, sys: 107 µs, total: 263 ms
Wall time: 205 ms


For a finner control, the torch pretrained model may also be retreived with `fundus_vessels_toolkit.models.segmentation_model( model_name )`.


### Display Segmentation Result


Displays the predicted segmentation and the ground truth side by side.

_(The prediction is on the left, the ground truth on the right, the middle show the false positive in green and the false negative in orange.)_


In [ ]:
grid = GridspecLayout(1, 3, height="600px")

for i in range(3):
    v = imshow(raw)
    grid[0, i] = v

grid[0, 0].add_label(vessels, "vessels", colormap="white", options={"opacity": 0.3})
grid[0, 1].add_label(vessels + 2 * vessels_true, "vessels", colormap={3: "white", 1: "green", 2: "orange"})
grid[0, 2].add_label(vessels_true, "vessels", colormap="white", options={"opacity": 0.3})
sync_views(*[grid[0, i] for i in range(3)])

grid